<img src="../docs/_static/logo-rsm.png" align="right" width="200">


# Part 1: Gradient Descent

This notebook studies inference for a ___linear model___ using gradient descent.

Our focus will be using `pytorch` to make the implementation more parsimonious:
1. Generate data for a linear model
1. Review gradient descent, implement inference in `numpy` (including coding the gradients)
1. Implement inference using PyTorch's autograd 🎉 
1. Leverage `torch.nn` module to simplify the implementation

In [1]:
import numpy as np
import torch

from src.common import print_result

<br>

## Data

Our data generating process is a linear model

$
\quad \mathbf y = a + b\, \mathbf x_1 + c\, \mathbf x_2 + d\, \mathbf x_3 + \mathbf\varepsilon
$

We'll set

- $a = 3$
- $b = -2$
- $c = 1$
- $d = -1$
- $\varepsilon \sim \mathcal{N}(\mu_{\varepsilon}, \sigma_{\varepsilon})$ with $\mu_{\varepsilon}=0$ and $\sigma_{\varepsilon}=1$


In [2]:
np.random.seed(1234)

# data size
N = 1_000

# parameters
at = 3
bt = -2
ct = 1
dt = -1
sigma_err = 1

# generate data
err = np.random.normal(0, sigma_err, N)
x1 = np.random.normal(2, 1, N)
x2 = np.random.normal(3, 1, N)
x3 = np.random.normal(1, 1, N)
y = at + bt * x1 + ct * x2 + dt * x3 + err

<br>

## Manual implementation with `numpy`

1. Manually implement loss function $\mathcal{L} = \frac 1 2 (\hat y - y)^2$
1. Manually calculate gradient $\partial_a \mathcal{L} = \hat y - y$
1. Manually update parameters $\quad a^{i+1} = a^{i} - \alpha \partial_a \mathcal{L}$

In [3]:
torch.manual_seed(1234)
np.random.seed(1234)

In [4]:
# define learning rate
learning_rate = 1e-3
n_epochs = 100_000

# random starting values
am, bm, cm, dm = np.random.uniform(-1, 1, 4)

# training: looping over epochs
for e in range(n_epochs):

    y_pred = am + bm * x1 + cm * x2 + dm * x3

    loss = np.mean((y_pred - y) ** 2 / 2)

    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss:.6f}")

    grad_y_pred = y_pred - y

    grad_a = (grad_y_pred * 1).mean()
    grad_b = (grad_y_pred * x1).mean()
    grad_c = (grad_y_pred * x2).mean()
    grad_d = (grad_y_pred * x3).mean()

    am -= learning_rate * grad_a  # am = am - learning_rate * grad_a
    bm -= learning_rate * grad_b
    cm -= learning_rate * grad_c
    dm -= learning_rate * grad_d

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")

    9999 | loss = 0.586737
   19999 | loss = 0.504840
   29999 | loss = 0.481684
   39999 | loss = 0.475138
   49999 | loss = 0.473287
   59999 | loss = 0.472763
   69999 | loss = 0.472615
   79999 | loss = 0.472574
   89999 | loss = 0.472562
   99999 | loss = 0.472558

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 2.90 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3


<br>

## Use `autograd` from `PyTorch`

1. Use `torch` gradients
1. Manually update parameters

In [5]:
torch.manual_seed(1234)
np.random.seed(1234)

### Some first steps

#### Turn `numpy` arrays into `torch` tensors

In [6]:
x1t = torch.tensor(x1)
x2t = torch.tensor(x2)
x3t = torch.tensor(x3)
yt = torch.tensor(y)

#### Initialize coefficients

In [7]:
a = torch.randn((), requires_grad=True)
b = torch.randn((), requires_grad=True)
c = torch.randn((), requires_grad=True)
d = torch.randn((), requires_grad=True)

#### Make prediction

In [8]:
yt_pred = a + b * x1t + c * x2t + d * x3t
yt_pred[:10]

tensor([-1.4373, -4.2672, -2.1062, -2.8174, -3.1186, -3.7332, -2.9870, -3.0341,
        -0.9524, -1.0748], dtype=torch.float64, grad_fn=<SliceBackward0>)

#### Calculate loss

In [9]:
loss = torch.mean((yt_pred - yt).pow(2) / 2)
loss

tensor(10.3387, dtype=torch.float64, grad_fn=<MeanBackward0>)

#### Backward pass and gradients

In [10]:
loss.backward()

In [11]:
a.grad.item(), (yt_pred - yt).mean().item()

(-2.996028184890747, -2.996028283779662)

In [12]:
b.grad.item(), ((yt_pred - yt) * x1t).mean().item()

(-3.8542213439941406, -3.854221284804097)

In [13]:
c.grad.item(), ((yt_pred - yt) * x2t).mean().item()

(-10.999500274658203, -10.999500625744313)

In [14]:
d.grad.item(), ((yt_pred - yt) * x3t).mean().item()

(-1.729453444480896, -1.7294534728842264)

### Putting it all together

In [15]:
torch.manual_seed(501)
np.random.seed(501)

In [16]:
learning_rate = 1e-3
n_epochs = 100_000

ap = torch.randn((), requires_grad=True)
bp = torch.randn((), requires_grad=True)
cp = torch.randn((), requires_grad=True)
dp = torch.randn((), requires_grad=True)

for e in range(n_epochs):

    # forward
    yt_pred = ap + bp * x1t + cp * x2t + dp * x3t
    loss = torch.mean((yt_pred - yt).pow(2) / 2)
    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss.item():.6f}")

    # backward
    loss.backward()

    with torch.no_grad():
        ap -= learning_rate * ap.grad
        bp -= learning_rate * bp.grad
        cp -= learning_rate * cp.grad
        dp -= learning_rate * dp.grad

        ap.grad = None
        bp.grad = None
        cp.grad = None
        dp.grad = None

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")
print_result([ap, bp, cp, dp], "PyTorch")

    9999 | loss = 0.499813
   19999 | loss = 0.480265
   29999 | loss = 0.474737
   39999 | loss = 0.473174
   49999 | loss = 0.472732
   59999 | loss = 0.472607
   69999 | loss = 0.472571
   79999 | loss = 0.472561
   89999 | loss = 0.472558
   99999 | loss = 0.472557

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 2.90 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3
PyTorch:  y = 2.91 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3


<br>

## Using `torch.nn`

1. Use `torch` loss function
1. Use `torch` gradients
1. Use `torch` parameter update

In [17]:
torch.manual_seed(501)
np.random.seed(501)

In [18]:
xt = torch.tensor(np.column_stack([x1, x2, x3]).astype(np.float32))
yt = torch.tensor(y.astype(np.float32))

In [19]:
model = torch.nn.Sequential(torch.nn.Linear(3, 1), torch.nn.Flatten(0, 1))

In [20]:
learning_rate = 1e-3
n_epochs = 100_000

loss_fn = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for e in range(n_epochs):

    # forward
    yt_pred = model(xt)
    loss = loss_fn(yt_pred, yt) / 2

    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss.item():.6f}")

    optimizer.zero_grad()
    # backward
    loss.backward()

    # udpate
    optimizer.step()

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")
print_result([ap, bp, cp, dp], "PyTorch")
print_result(torch.cat([model[0].bias, model[0].weight.flatten()]), "PyTorch2")

    9999 | loss = 0.519029
   19999 | loss = 0.485698
   29999 | loss = 0.476274
   39999 | loss = 0.473609
   49999 | loss = 0.472855
   59999 | loss = 0.472641
   69999 | loss = 0.472581
   79999 | loss = 0.472564
   89999 | loss = 0.472559
   99999 | loss = 0.472558

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 2.90 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3
PyTorch:  y = 2.91 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3
PyTorch2: y = 2.91 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3


<br>
<br>
&mdash; <br>
Sebastian Gabel <br>
Rotterdam School of Management <br>

www.github.com/sbstn-gbl <br>
www.sebastiangabel.com <br>

<br>

Inspired by this [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)

### Some first steps

#### Turn `numpy` arrays into `torch` tensors

In [21]:
x1t = torch.tensor(x1)
x2t = torch.tensor(x2)
x3t = torch.tensor(x3)
yt = torch.tensor(y)

#### Initialize coefficients

In [22]:
a = torch.randn((), requires_grad=True)
b = torch.randn((), requires_grad=True)
c = torch.randn((), requires_grad=True)
d = torch.randn((), requires_grad=True)

#### Make prediction

In [23]:
yt_pred = a + b * x1t + c * x2t + d * x3t
yt_pred[:10]

tensor([0.5723, 2.4670, 2.9830, 2.0257, 6.6208, 2.1603, 6.0776, 4.6468, 2.9855,
        2.5320], dtype=torch.float64, grad_fn=<SliceBackward0>)

#### Calculate loss

In [24]:
loss = torch.mean((yt_pred - yt).pow(2) / 2)
loss

tensor(9.8533, dtype=torch.float64, grad_fn=<MeanBackward0>)

#### Backward pass and gradients

In [25]:
loss.backward()

In [26]:
a.grad.item(), (yt_pred - yt).mean().item()

(2.4855661392211914, 2.4855661035428267)

In [27]:
b.grad.item(), ((yt_pred - yt) * x1t).mean().item()

(8.250786781311035, 8.250786465544447)

In [28]:
c.grad.item(), ((yt_pred - yt) * x2t).mean().item()

(7.396797180175781, 7.3967969924699934)

In [29]:
d.grad.item(), ((yt_pred - yt) * x3t).mean().item()

(3.93881893157959, 3.938818870680674)

### Putting it all together

In [30]:
torch.manual_seed(501)
np.random.seed(501)

In [31]:
learning_rate = 1e-3
n_epochs = 100_000

ap = torch.randn((), requires_grad=True)
bp = torch.randn((), requires_grad=True)
cp = torch.randn((), requires_grad=True)
dp = torch.randn((), requires_grad=True)

for e in range(n_epochs):

    # forward
    yt_pred = ap + bp * x1t + cp * x2t + dp * x3t
    loss = torch.mean((yt_pred - yt).pow(2) / 2)
    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss.item():.6f}")

    # backward
    loss.backward()

    with torch.no_grad():
        ap -= learning_rate * ap.grad
        bp -= learning_rate * bp.grad
        cp -= learning_rate * cp.grad
        dp -= learning_rate * dp.grad

        ap.grad = None
        bp.grad = None
        cp.grad = None
        dp.grad = None

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")
print_result([ap, bp, cp, dp], "PyTorch")

    9999 | loss = 0.499813
   19999 | loss = 0.480265
   29999 | loss = 0.474737
   39999 | loss = 0.473174
   49999 | loss = 0.472732
   59999 | loss = 0.472607
   69999 | loss = 0.472571
   79999 | loss = 0.472561
   89999 | loss = 0.472558
   99999 | loss = 0.472557

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 2.90 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3
PyTorch:  y = 2.91 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3


In [32]:
torch.manual_seed(501)
np.random.seed(501)

In [33]:
xt = torch.tensor(np.column_stack([x1, x2, x3]).astype(np.float32))
yt = torch.tensor(y.astype(np.float32))

In [34]:
model = torch.nn.Sequential(torch.nn.Linear(3, 1), torch.nn.Flatten(0, 1))

In [35]:
learning_rate = 1e-3
n_epochs = 100_000

loss_fn = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for e in range(n_epochs):

    # forward
    yt_pred = model(xt)
    loss = loss_fn(yt_pred, yt) / 2

    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss.item():.6f}")

    optimizer.zero_grad()
    # backward
    loss.backward()

    # udpate
    optimizer.step()

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")
print_result([ap, bp, cp, dp], "PyTorch")
print_result(torch.cat([model[0].bias, model[0].weight.flatten()]), "PyTorch2")

    9999 | loss = 0.519029
   19999 | loss = 0.485698
   29999 | loss = 0.476274
   39999 | loss = 0.473609
   49999 | loss = 0.472855
   59999 | loss = 0.472641
   69999 | loss = 0.472581
   79999 | loss = 0.472564
   89999 | loss = 0.472559
   99999 | loss = 0.472558

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 2.90 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3
PyTorch:  y = 2.91 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3
PyTorch2: y = 2.91 + -1.99 x_1 + 1.04 x_2 + -1.02 x_3


<br>
<br>
&mdash; <br>
Sebastian Gabel <br>
Rotterdam School of Management <br>

www.github.com/sbstn-gbl <br>
www.sebastiangabel.com <br>

<br>

Inspired by this [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)